In [2]:
from PIL.Image import alpha_composite
import torch
from torchvision import transforms

In [3]:
'''
In principle the pipeline should contain the following functions:

1. Depth map to Zero-one depth tensor
Input :  (512,512) depth map
Output:  (512,512,d) depth tensor
Implementation: create_depth_tensor

2. Zero-One depth tensor to smoothed depth tensor
Input :  (512,512,d) depth tensor
Output:  (512,512,d) depth tensor
Implementation: naive_alpha

3. Depth tensor and image to RGBA tensor
Input :  (512,512,d) depth tensor
         (3,512,512) RGB image
Output:  (4,512,512,d) RGBA tensor
Implementation: rgba

4. RGBA tensor homography warp to target pose
Input :  (4,512,512,d) RGBA tensor
          Target Pose
Output:  (4,512,512,d) RGBA tensor warped to target pose
Implementation: MISSING

5. RGBA tensor alpha composite to RGB image
Input :  (4,512,512,d) RGBA tensor
Output:  (4,512,512) RGBA image, 
Implementation: back_to_front_alpha_composite

6. RGB images + alphas to blended output RGB image (Equation 8)
Input :  5 (4,512,512) RGBA images
Output:  (3,512,512) RGB image
Implementation: 

'''

'\nIn principle the pipeline should contain the following functions:\n\n1. Depth map to Zero-one depth tensor\nInput :  (512,512) depth map\nOutput:  (512,512,d) depth tensor\nImplementation: \n\n2. Zero-One depth tensor to smoothed depth tensor\nInput :  (512,512,d) depth tensor\nOutput:  (512,512,d) depth tensor\nImplementation: naive_alpha\n\n3. Depth tensor and image to RGBA tensor\nInput :  (512,512,d) depth tensor\n         (3,512,512) RGB image\nOutput:  (4,512,512,d) RGBA tensor\nImplementation: rgba\n\n4. RGBA tensor homography warp to target pose\nInput :  (4,512,512,d) RGBA tensor\n          Target Pose\nOutput:  (4,512,512,d) RGBA tensor warped to target pose\nImplementation: MISSING\n\n5. RGBA tensor alpha composite to RGB image\nInput :  (4,512,512,d) RGBA tensor\nOutput:  (4,512,512) RGBA image, \nImplementation: back_to_front_alpha_composite\n\n6. RGB images + alphas to blended output RGB image (Equation 8)\nInput :  5 (4,512,512) RGBA images\nOutput:  (3,512,512) RGB i

In [126]:
# Input  : (512,512) depth_map
#          Integer layers          
# Output : (512,512, layers) depth tensor

def create_depth_tensor(depth_map, layers):
    assert depth_map.shape == (512,512), 'Depth map needs to be shape (512,512)'
    assert type(layers) == int, 'layers needs to be integer'
    
    #get the min_depth and the max_depth
    min_depth = torch.min(depth_map)
    max_depth = torch.max(depth_map)*1.000001
    
    #shift all depths by the min_depth
    depth_rel = depth_map-min_depth
    
    #get the bin_size
    bin_size  = (max_depth-min_depth)/layers
    
    #get the depth_layer for every pixel
    depth_layers = (depth_rel/bin_size).int()
    
    #convert to 3d 1-hot-encoding depth_tensor (512,512,layers)
    depth_tensor = (torch.arange(layers) == depth_layers[...,None]).int()
    
    return depth_layers, depth_tensor 

In [127]:
# Test the function create_depth_tensor

depth_map = torch.rand((512,512))
layers = 10

layer, tensor = create_depth_tensor(depth_map, layers)
print(tensor.shape)
for i in range(5):
    pixel = torch.randint(0,512,size=(2,))
    print('Testpixel',i)
    print('Depth map value:',depth_map[pixel[0],pixel[1]])
    print('Depth layer:',layer[pixel[0],pixel[1]])
    print('Depth tensor:',tensor[pixel[0],pixel[1]])
    print('---------------------------')
    
    
# Test all pixels
a=torch.argmax (tensor, dim=2)
#torch.allclose( a, layer)
b=layer - a
for i in range(512):
    for j in range(512):
        if b[i,j] != 0:
            print(i,j)
            print(depth_map[i,j])
            print(layer[i,j])
            print(a[i,j])
            print(tensor[i,j])
b[b != 0]

torch.Size([512, 512, 10])
Testpixel 0
Depth map value: tensor(0.3427)
Depth layer: tensor(3, dtype=torch.int32)
Depth tensor: tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
---------------------------
Testpixel 1
Depth map value: tensor(0.0210)
Depth layer: tensor(0, dtype=torch.int32)
Depth tensor: tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
---------------------------
Testpixel 2
Depth map value: tensor(0.2487)
Depth layer: tensor(2, dtype=torch.int32)
Depth tensor: tensor([0, 0, 1, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
---------------------------
Testpixel 3
Depth map value: tensor(0.9582)
Depth layer: tensor(9, dtype=torch.int32)
Depth tensor: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=torch.int32)
---------------------------
Testpixel 4
Depth map value: tensor(0.0380)
Depth layer: tensor(0, dtype=torch.int32)
Depth tensor: tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=torch.int32)
---------------------------


tensor([], dtype=torch.int64)

In [113]:
# takes a depth tensor of shape (512,512,depth) as input
# input is assumed to assign every pixel of the (512,512) image to exactly one depth
# output is a (512,512,depth) tensor with smoothed depth information
# smoothes depth channel with filter [0.1 , 0.2 , 0.4 , 0.2 , 0.1]

def naive_alpha(depth_tensor):
    
    assert len(depth_tensor.shape)==3, 'Depth tensor needs to be 512x512xdepth'
    depth = depth_tensor.shape[2]
    assert depth_tensor.shape == (512,512,depth), 'Depth tensor needs to be 512x512xdepth'
    assert torch.allclose(torch.sum(depth_tensor, dim=2), torch.ones((512,512))), 'Depth tensor not normalized'
    
    # Note: Add 4 units of 0-padding in depth direction for the boundary conditions
    zero_padding = torch.zeros((512,512,2))
    alpha        = torch.cat((zero_padding, depth_tensor, zero_padding), dim=2)
    
    # Naively distribute the alpha=1 value over its neighborhood
    # Equivalent to a 1D-convolution of the depth dimension with the filter [0.1 , 0.2 , 0.4 , 0.2 , 0.1]  ??
    alpha = alpha[:,:,0:-4]*0.1 + alpha[:,:,1:-3]*0.2 + alpha[:,:,2:-2]*0.4 + alpha[:,:,3:-1]*0.2 + alpha[:,:,4:]*0.1

    
    # Assure normalization
    # Note: In case the significant depth is at the boundary, this is necessary ??
    sum_along_depth = torch.sum(alpha, dim=2)
    alpha           = alpha / sum_along_depth.unsqueeze(2)
    
    assert alpha.shape == (512,512,depth), 'Alpha Dimensionality failed'
    assert torch.allclose(torch.sum(alpha, dim=2), torch.ones((512,512))), 'Alpha Normalization failed'

    
    return alpha


#ToDo
       
# Boundary conditions: Something else than 0-padding?
    # mpi[:,:,0]  =  mpi[:,:,0]  * 0.5  + mpi[:,:,1]  * 0.5
    # mpi[:,:,1]  =  mpi[:,:,0]  * 0.25 + mpi[:,:,1]  * 0.5 + mpi[:,:,2]  * 0.25
    
    # mpi[:,:,-1] =  mpi[:,:,-1] * 0.5  + mpi[:,:,-2] * 0.5
    # mpi[:,:,-2] =  mpi[:,:,-1] * 0.25 + mpi[:,:,-2] * 0.5 + mpi[:,:,-3] * 0.25 

    
# Image size (512,512) to arbitrary


# Smoothing kernel as input parameter

In [4]:
# Test the naive_alpha function for depth=16 

one_layer = torch.ones((512,512,1))
zero_layer= torch.zeros((512,512,1))

zero_6layer = torch.zeros((512,512,6))
zero_9layer = torch.zeros((512,512,9))
zero_14layer= torch.zeros((512,512,14))
zero_15layer= torch.zeros((512,512,15))

# Test having the significant pixel at different depths to see boundary condition behaviour
test_0 = torch.cat((one_layer, zero_15layer) , dim=2)
test_1 = torch.cat((zero_layer, one_layer, zero_14layer), dim=2)
test_7 = torch.cat((zero_6layer, one_layer, zero_9layer), dim=2)
test_14= torch.cat((zero_14layer,one_layer , zero_layer), dim=2)
test_15= torch.cat((zero_15layer, one_layer ), dim=2)

In [5]:
naive_alpha(test_0)[0,0]

tensor([0.5714, 0.2857, 0.1429, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [6]:
naive_alpha(test_1)[0,0]

tensor([0.2222, 0.4444, 0.2222, 0.1111, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [7]:
naive_alpha(test_7)[0,0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.1000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000])

In [8]:
naive_alpha(test_14)[0,0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.1111, 0.2222, 0.4444, 0.2222])

In [9]:
naive_alpha(test_15)[0,0]

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.1429, 0.2857, 0.5714])

In [10]:
def rgba(image, depth_tensor):
    
    # Input: RGB image 3 x 512 x 512 
    # Input: Depth tensor 512 x 512 x Depth
    # Output: RGB-alpha depth image 4 x 512 x 512 x Depth
    
    assert len(depth_tensor.shape)==3, 'Depth tensor needs to be 512x512xdepth'
    Depth = depth_tensor.shape[2]
    assert depth_tensor.shape == (512,512,Depth), 'Depth tensor needs to be 512x512xdepth'
    assert torch.allclose(torch.sum(depth_tensor, dim=2), torch.ones((512,512))), 'Depth tensor not normalized'
    
    assert len(image.shape)==3, 'Image must be 3x512x512'
    assert image.shape[0]==3, 'Image must be 3x512x512'

    
    output=torch.zeros((4,512,512,Depth))
    
    for d in range(Depth):
                
        output[:,:,:,d]=torch.stack([image[0,:,:],image[1,:,:],image[2,:,:],depth_tensor[:,:,d]], dim=0)
                
    return output
            

In [11]:
# Test the rgba implementation with a random image
# test_depth tensor has one layer depth=6 where alpa=1, alpha=0 everywhere else

test_image = torch.rand((3,512,512))

test_depth_tensor = torch.cat((zero_6layer, one_layer, zero_9layer), dim=2)

test_rgba = rgba(test_image, test_depth_tensor)

# Look at test results for random pixels
pixel1 = torch.randint(0, 255, size=(2,))
print('Random pixel1 :', pixel1)
print('RGB values of test image at pixel1 :', test_image[:,pixel1[0],pixel1[1]])
print('Alpha values along depth at pixel1 :', test_depth_tensor[pixel1[0],pixel1[1]])
print('RGBa value at pixel1 at depth0 :', test_rgba[:,pixel1[0],pixel1[1],0])
print('RGBa value at pixel1 at depth6 :', test_rgba[:,pixel1[0],pixel1[1],6])
print('RGBA shape:', test_rgba.shape)

print('----------------------------------------')

pixel2 = torch.randint(0, 255, size=(2,))
print('Random pixel2 :', pixel2)
print('RGB values of test image at pixel2 :', test_image[:,pixel2[0],pixel2[1]])
print('Alpha values along depth at pixel2 :', test_depth_tensor[pixel2[0],pixel2[1]])
print('RGBa value at pixel2 at depth0 :', test_rgba[:,pixel2[0],pixel2[1],0])
print('RGBa value at pixel2 at depth6 :', test_rgba[:,pixel2[0],pixel2[1],6])
print('RGBA shape:', test_rgba.shape)

Random pixel1 : tensor([234, 115])
RGB values of test image at pixel1 : tensor([0.1573, 0.4703, 0.9468])
Alpha values along depth at pixel1 : tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
RGBa value at pixel1 at depth0 : tensor([0.1573, 0.4703, 0.9468, 0.0000])
RGBa value at pixel1 at depth6 : tensor([0.1573, 0.4703, 0.9468, 1.0000])
RGBA shape: torch.Size([4, 512, 512, 16])
----------------------------------------
Random pixel2 : tensor([197,  86])
RGB values of test image at pixel2 : tensor([0.5164, 0.9373, 0.7269])
Alpha values along depth at pixel2 : tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
RGBa value at pixel2 at depth0 : tensor([0.5164, 0.9373, 0.7269, 0.0000])
RGBa value at pixel2 at depth6 : tensor([0.5164, 0.9373, 0.7269, 1.0000])
RGBA shape: torch.Size([4, 512, 512, 16])


In [12]:
# Test the rgba implementation with a random image
# test_depth_tensor2 is the naive_alpha output of test_depth_tensor 

test_image2 = torch.rand((3,512,512))

test_depth_tensor2 = naive_alpha(test_depth_tensor)

test_rgba2 = rgba(test_image2, test_depth_tensor2)

# Look at test results for random pixels
pixel1 = torch.randint(0, 255, size=(2,))
print('Random pixel1 :', pixel1)
print('RGB values of test image at pixel1 :', test_image[:,pixel1[0],pixel1[1]])
print('Alpha values along depth at pixel1 :', test_depth_tensor[pixel1[0],pixel1[1]])
print('RGBa value at pixel1 at depth0 :', test_rgba[:,pixel1[0],pixel1[1],0])
print('RGBa value at pixel1 at depth4 :', test_rgba[:,pixel1[0],pixel1[1],4])
print('RGBa value at pixel1 at depth6 :', test_rgba[:,pixel1[0],pixel1[1],6])
print('RGBA shape:', test_rgba.shape)

print('----------------------------------------')

pixel2 = torch.randint(0, 255, size=(2,))
print('Random pixel2 :', pixel2)
print('RGB values of test image at pixel2 :', test_image[:,pixel2[0],pixel2[1]])
print('Alpha values along depth at pixel2 :', test_depth_tensor[pixel2[0],pixel2[1]])
print('RGBa value at pixel2 at depth0 :', test_rgba[:,pixel2[0],pixel2[1],0])
print('RGBa value at pixel2 at depth4 :', test_rgba[:,pixel2[0],pixel2[1],4])
print('RGBa value at pixel2 at depth6 :', test_rgba[:,pixel2[0],pixel2[1],6])
print('RGBA shape:', test_rgba.shape)

Random pixel1 : tensor([143, 124])
RGB values of test image at pixel1 : tensor([0.2739, 0.2924, 0.7183])
Alpha values along depth at pixel1 : tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
RGBa value at pixel1 at depth0 : tensor([0.2739, 0.2924, 0.7183, 0.0000])
RGBa value at pixel1 at depth4 : tensor([0.2739, 0.2924, 0.7183, 0.0000])
RGBa value at pixel1 at depth6 : tensor([0.2739, 0.2924, 0.7183, 1.0000])
RGBA shape: torch.Size([4, 512, 512, 16])
----------------------------------------
Random pixel2 : tensor([206,  37])
RGB values of test image at pixel2 : tensor([0.5064, 0.9441, 0.9367])
Alpha values along depth at pixel2 : tensor([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
RGBa value at pixel2 at depth0 : tensor([0.5064, 0.9441, 0.9367, 0.0000])
RGBa value at pixel2 at depth4 : tensor([0.5064, 0.9441, 0.9367, 0.0000])
RGBa value at pixel2 at depth6 : tensor([0.5064, 0.9441, 0.9367, 1.0000])
RGBA shape: torch.Size([4, 512, 512, 16])


In [13]:
# Function to alpha_composite a 4d rgb-alpha depth image from back to front
# Input  (4,512,512,Depth)
# Output (4,512,512)

def back_to_front_alphacomposite(rgba_depth_image):
    
    assert len(rgba_depth_image.shape)==4 , 'Input image needs to have shape 4 x 512 x 512 x Depth'
    assert rgba_depth_image.shape[0] == 4 , 'Input image needs to have shape 4 x 512 x 512 x Depth'
    Depth = rgba_depth_image.shape[3] 
    
    img = transforms.ToPILImage('RGBA')(rgba_depth_image[:,:,:,-1])

    for d in reversed(range(1,Depth)):
        
        layer = rgba_depth_image[:,:,:,d-1]
        layer = transforms.ToPILImage('RGBA')(layer)
        
        img = alpha_composite( layer , img)

    img = transforms.ToTensor()(img)
    return img


In [17]:
a=back_to_front_alphacomposite(test_rgba2)
print(test_rgba2[:,0,0])
print(a[:,0,0])

tensor([[0.7324, 0.7324, 0.7324, 0.7324, 0.7324, 0.7324, 0.7324, 0.7324, 0.7324,
         0.7324, 0.7324, 0.7324, 0.7324, 0.7324, 0.7324, 0.7324],
        [0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220,
         0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220, 0.0220],
        [0.9619, 0.9619, 0.9619, 0.9619, 0.9619, 0.9619, 0.9619, 0.9619, 0.9619,
         0.9619, 0.9619, 0.9619, 0.9619, 0.9619, 0.9619, 0.9619],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.1000, 0.2000, 0.4000, 0.2000, 0.1000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
tensor([0.7294, 0.0196, 0.9608, 0.6902])
